# "Fazendo uma Pirâmide Etária com Plotly"
> "Rumo ao domínio dessa ferramenta arrojada!"

- toc: true
- branch: master
- badges: false
- comments: true
- author: Augusto dos Santos Pereira
- categories: [Plotly, Gráficos, Python]

In [1]:
# hide_input
# This cell is required for the export to HTML to work.
import plotly.io as pio
#pio.renderers.default = 'plotly_mimetype+notebook'
pio.renderers.default = 'colab'
# Default is plotly_mimetype+notebook, but jekyll fails to parse plotly_mimetype.
#pio.renderers.default = 'notebook_connected'
# Uncomment below to avoid using a CDN for plotly.js
# pio.renderers.default = 'notebook'

# Inject the missing require.js dependency.
from IPython.display import display, HTML
#js = '<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js" integrity="sha512-c3Nl8+7g4LMSTdrm621y7kf9v3SDPnhxLNhcjFJbKECVnmZHTdo+IRO05sNLTH/D3vA6u1X32ehoLC7WFVdheg==" crossorigin="anonymous"></script>'
#display(HTML(js))

## A Pirâmide Etária


Frequentemente, nos meus diagnósticos para **Planejamento Urbano e Regional**, preciso fazer análise da **estrutura etária da população**, para identificar demandas por creches, escolas, trabalho e atendimento de saúde para idosos. Esse tipo de trabalho é realizado com base nas pirâmides etárias. Dada a importância desse tipo de gráfico nas análises populacionais, adiante demonstrarei como elaborá-los com Python, utilizando o pacote [PLotly](https://plotly.com/python/).

> Tip: Veja [aqui um ótimo vídeo do IBGE](https://www.youtube.com/watch?v=UPgR_LL0Fz0) sobre pirâmides etárias.

## Obtenção e Tratamento dos Dados

Os dados sobre população segundo faixas etárias foram obtidos na base [SIDRA](https://sidra.ibge.gov.br/Tabela/200) do IBGE armazenados no arquivo estruturaetaria.csv. Vamos carregar os dados em Pandas DataFrame e observar o seu conteúdo.

In [2]:
import pandas as pd # importando o pacote pandas

df_estrutura_etaria = pd.read_csv('../assets/data/estruturaetaria.csv', sep=';') # carregando os dados do CSV
df_estrutura_etaria # visualização dos dados importados

,Codmun,Sexo,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos ou mais
0,4125506,Masculino,10584,10836,12340,11664,11700,12533,12268,10940,9539,8012,6468,4825,3576,2187,1486,945,695
1,4125506,Feminino,10269,10483,11945,11636,12055,12834,12195,10928,10021,8334,6692,5508,3715,2642,1906,1068,1382


Podemos ver que a tabela se encontra em formato largo, com cada variável, faixa etária, representada por uma coluna. Para trabalharmos com Plotly, é mais adequado transformarmos o DataFrame em formato longo, com uma coluna chamada faixa, que conterá cada faixa etária correspondente ao respectivo montante populacional. Para tanto, podemos utilizar o método _melt_, para fazer essas alterações.

In [3]:
df_estrutura_etaria = df_estrutura_etaria.melt(id_vars=['Codmun', 'Sexo'], var_name='Faixa', value_name='População')

df_estrutura_etaria.head(10) # chamando as dez primeiras linhas

,Codmun,Sexo,Faixa,População
0,4125506,Masculino,0 a 4 anos,10584
1,4125506,Feminino,0 a 4 anos,10269
2,4125506,Masculino,5 a 9 anos,10836
3,4125506,Feminino,5 a 9 anos,10483
4,4125506,Masculino,10 a 14 anos,12340
5,4125506,Feminino,10 a 14 anos,11945
6,4125506,Masculino,15 a 19 anos,11664
7,4125506,Feminino,15 a 19 anos,11636
8,4125506,Masculino,20 a 24 anos,11700
9,4125506,Feminino,20 a 24 anos,12055


É conveniente também separarmos os dados do sexo masculino e do sexo feminino em diferentes DataFrames.

In [4]:
df_estrutura_etaria_f = df_estrutura_etaria[df_estrutura_etaria['Sexo'] == 'Feminino']
df_estrutura_etaria_m = df_estrutura_etaria[df_estrutura_etaria['Sexo'] == 'Masculino']

In [5]:
df_estrutura_etaria_f.head(10)

,Codmun,Sexo,Faixa,População
1,4125506,Feminino,0 a 4 anos,10269
3,4125506,Feminino,5 a 9 anos,10483
5,4125506,Feminino,10 a 14 anos,11945
7,4125506,Feminino,15 a 19 anos,11636
9,4125506,Feminino,20 a 24 anos,12055
11,4125506,Feminino,25 a 29 anos,12834
13,4125506,Feminino,30 a 34 anos,12195
15,4125506,Feminino,35 a 39 anos,10928
17,4125506,Feminino,40 a 44 anos,10021
19,4125506,Feminino,45 a 49 anos,8334


## Criando os Gráficos de Estrutura Etária Separadamente 

O primeiro passo a fazer é importar a função make_subplots, pois faremos um gráfico com dois eixos. Também é preciso importar plotly express, API de alto nível de abstração que permite fazer gráficos com apenas uma função, além de Plotly Graph Objects, subpacote que apresenta maior de

In [6]:
from plotly.subplots import make_subplots #importando a função make_subplots
import plotly.express as px # importando plotly express

A lógica que vamos adotar para produzir esse gráfico é produzir dois gráficos distintos e colocá-los em justaposição, lado a lado, como subplots de uma outra figura. Para tanto, vamos fazer o primeiro gráfico de barras, com a função bar do subpacote plotly express. Essa função apresenta vários argumentos nos quais podemos avançar significativamente na empreitada. Vamos indicar que o Data Frame a ser considerado deve ser aquele para a estrutura etária feminina, que a coluna para o eixo x deve ser População, a coluna para o eixo y deve ser Faixa, as cores devem ser associadas à coluna Sexo, a orientação do gráfico deve ser horizontal, a cor para o sexo feminino deve ser vermelha, segundo padrão rgb, e o modo das barras deve ser de sobreposição, overlay.

In [7]:
plot_fem = px.bar( # função para criar um gráfico de barras
    data_frame=df_estrutura_etaria_f # indicação de do DataFrame a ser considerado para fazer o plot
    , x='População'  # associação da coluna População ao eixo x
    , y='Faixa' # associação da coluna Faixa ao eixo y
    , color='Sexo' # associação da coluna Sexo a cor
    , orientation='h' # seleção da orientação horizontal
    , color_discrete_map={'Feminino':'rgb(228, 26,28)'} # mapeamento da cor vermelha ao sexo feminino
    , barmode='overlay') # escolha de barras em modo sobreposto

plot_fem.show() # fazendo a renderização do gráfico com o método show

Agora vamos fazer o mesmo para os dados da população de sexo masculino.

In [8]:
plot_masc = px.bar( # função para criar um gráfico de barras
    data_frame=df_estrutura_etaria_m # indicação de do DataFrame a ser considerado para fazer o plot
    , x='População' # associação da coluna População ao eixo x
    , y='Faixa' # associação da coluna Faixa ao eixo y
    , color='Sexo' # associação da coluna Sexo a cor
    , orientation='h' # seleção da orientação horizontal
    , barmode='overlay') # escolha de barras em modo sobreposto

plot_masc.show()

## Gerando Uma Figura com Espaços para Subplots

Acima temos dois gráficos separados, mas precisamos que eles estejam lado a lado para uma adequada visualização da estrutura etária. Assim, vamos utilizar a função make_suplots, que gera uma figura com espaços separados em linhas e colunas para que possam ser associados eixos de gráficos, os subplots.

Na função, vamos indicar que queremos espaço para dois subplots, em uma única linha, argumento rows, dividida em duas colunas, argumento cols. Também é preciso que o argumento shared_yaxes atribuído o valor True, o que indica que os dois subplots compartilharão um único eixo vertical. Por fim, precisamos estabelecer que não deve haver espaçamento entre ambos os subplots - horizontal_spacing=0.

In [9]:
fig = make_subplots( # Função para fazer uma figura com subplots
    rows=1 # indicação que a figura deve ter uma linha para subplots
    , cols=2 # indicação que a figura deve ter duas colunas para subplots
    , shared_yaxes=True # indicação de que os subplots deve comparilhar um único eixo vertical, y
#    , shared_xaxes=True
    , horizontal_spacing=0) # indicação de que o espaçamento horizontal entre ambas os plots deve ser zero

Quando pedimos para Plotly renderizar essa figura vazia, ela mostra um dos espaços reservado para um dos subplots.

In [10]:
fig.show()

## Inserindo os Gráficos como Subplots


Precisamos inserir os dois plots elaborados primeiramente, plot_fem e plot_masc, no espaço criado para elas na figura fig. Para podermos fazer isso, precisamos entender a estrutura desses objetos. Vamos começar com a figura fig.

In [11]:
 print(fig)

Figure({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 0.5]},
               'xaxis2': {'anchor': 'y2', 'domain': [0.5, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0]},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 1.0], 'matches': 'y', 'showticklabels': False}}
})


Notamos acima que fig é um objeto Plot Figure, uma série de instruções em forma de árvore, como um dicionário, que apresenta nos dados, data, uma lista vazia, ou seja, nenhum dado, ao passo que tem um layout com espaço para dois subplots, o primeiro formado pelos eixos x e y, e o segundo formado pelos eixos x2 e y2. O primeiro subplot toma metade do espaçamento horizontal da figura (0.0 a 0.5), e o segundo indo de 0.5, metade do eixo horizontal, até o o ponto mais a direita, 1.0. Por essa razão, o que precisamos inserir aqui são os valores que devem ir em data, ou seja, o mapeamento dos dados aos respectivos eixo e cores.

Vejamos agora o que é o objeto plot_fem

In [12]:
print(plot_fem)

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Sexo=Feminino<br>População=%{x}<br>Faixa=%{y}<extra></extra>',
              'legendgroup': 'Feminino',
              'marker': {'color': 'rgb(228, 26,28)', 'opacity': 0.5},
              'name': 'Feminino',
              'offsetgroup': 'Feminino',
              'orientation': 'h',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([10269, 10483, 11945, 11636, 12055, 12834, 12195, 10928, 10021,  8334,
                           6692,  5508,  3715,  2642,  1906,  1068,  1382], dtype=int64),
              'xaxis': 'x',
              'y': array(['0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
                          '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos',
                          '40 a 44 anos', '45 a 49 anos', '50 a 54 anos', '55 a 59 anos',
                          '60 a 64 anos', '65 a 69 anos

Nota-se que o plot_fem também é um objeto Plotly Figure, com especificação de data e de layout. Por essa razão, não podemos simplesmente inserir plot_fem como um subplot de fig. Cada um apresenta seu próprio layout. O que precisamos é passar apenas os valores da chave data, que pode ser acessada pela notação de recuperação de dados de dicionário plot_fem\['data'\]\[0\]. Fazemos isso, com o método add_traces, chamado no objeto fig, indicando em que linha, row, e coluna, esses traços, valores de data, devem ser inseridos.

In [13]:
fig.add_traces(plot_fem['data'][0], rows=1, cols=1);
fig.add_traces(plot_masc['data'][0], rows=1, cols=2);

In [14]:
fig.show()

## Ajustes Finais

Estamos quase lá. Precisamos fazer alguns ajustes. O eixo x da primeira figura precisa ter seu sentido revertido.

In [15]:
fig.layout.xaxis.autorange = 'reversed'
fig.show()

Precisamos indicar que as barras nao podem estar agrupadas em diferentes espaços, mas sobrepostas, em overlay.

Podemos observar o podemos observar o esquema da figura para saber como fazer a alteração.

In [16]:
fig.layout.barmode = 'overlay'

fig.show()

Agora podemos tirar as legendas, inserir títulos de eixos x, inserir o título do gráfico

In [17]:
fig.data[0].showlegend= False
fig.data[1].showlegend= False

fig.layout.xaxis.title.text = 'Feminino'
fig.layout.xaxis2.title.text = 'Masculino'

fig.layout.title.text = f'<b>Pirâmide Etária<b>'

fig.show()